Predict the burned area of forest fires, in the northeast region of Portugal, by using meteorological and other data. Data Analysis, Data visualization, Feature Selection, about 10 Machine Learning models/estimators. Multilayer Perceptron(Deep Learning/Artificial Neural Network). Cross validation is used order to avoid overfitting.

In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor

from sklearn.metrics import mean_absolute_error, accuracy_score

from sklearn.model_selection import train_test_split


In [2]:
# load dataset
dataframe = pd.read_csv(r"../input/forestfires.csv")

# Encode Data
dataframe.month.replace(('jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec'),(1,2,3,4,5,6,7,8,9,10,11,12), inplace=True)
dataframe.day.replace(('mon','tue','wed','thu','fri','sat','sun'),(1,2,3,4,5,6,7), inplace=True)

In [3]:
dataframe["fire"] = (dataframe["area"] > 0).astype(int)

dataframe.sample(10)

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,fire
406,6,5,9,6,87.1,291.3,860.6,4.0,17.0,67,4.9,0.0,3.95,1
156,2,4,9,6,93.4,145.4,721.4,8.1,28.6,27,2.2,0.0,1.61,1
479,7,4,7,1,89.2,103.9,431.6,6.4,22.6,57,4.9,0.0,278.53,1
497,3,4,8,2,96.1,181.1,671.2,14.3,32.3,27,2.2,0.0,14.68,1
33,6,3,9,7,91.7,75.6,718.3,7.8,17.7,39,3.6,0.0,0.00,0
457,1,4,8,3,91.7,191.4,635.9,7.8,19.9,50,4.0,0.0,82.75,1
135,3,5,8,6,93.5,139.4,594.2,20.3,17.6,52,5.8,0.0,0.00,0
452,7,4,8,1,91.5,238.2,730.6,7.5,17.7,65,4.0,0.0,0.00,0
366,4,5,9,2,91.1,132.3,812.1,12.5,15.9,38,5.4,0.0,1.75,1
239,7,5,4,7,81.9,3.0,7.9,3.5,13.4,75,1.8,0.0,0.00,0


In [4]:
X = dataframe[["X", "Y", "month", "day", "FFMC", "DMC", "DC", "ISI", "temp", "RH", "wind", "rain"]]
Y = dataframe["fire"]

In [5]:
Y = np.array(Y).reshape((len(Y), 1))
print(Y.shape)

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

(517, 1)


'Wind', 'RH' and 'DMC' were top 3 selected features/feature combination for predicting 'Area'
 using Recursive Feature Elimination, the 2nd selected feature was atually one of the attributes with the highest correlation with the 
 'Area'


In [6]:
# evaluate model with standardized dataset
from xgboost import XGBClassifier

estimator =  XGBClassifier(
 learning_rate =0.01,
 n_estimators=280,
 max_depth=7,
 min_child_weight=2,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
estimator.fit(x_train,y_train)  
predictions = estimator.predict(x_test)

# Evaluate the model
score = accuracy_score(predictions, y_test)

from sklearn.externals import joblib
joblib.dump(estimator, "estimator.joblib")


print(score)

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.5576923076923077


/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
